In [1]:
import multiprocessing
import random
from collections import defaultdict, OrderedDict
from pprint import pformat
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
from deap import algorithms, base, creator, tools

%matplotlib notebook

## Problem and Solution Definitions

### Class that holds the problem definition.

This class must implements:
* A method to represent the problem graphically (`__str__`)
* A method to generate a problem instance from an input file (`from_file`).

In [2]:
class Problem:
    def __init__(self, rows: int, cols: int, fleet: int, rides: List[List[int]], bonus: int, steps: int):
        self.rows = rows
        self.cols = cols
        self.fleet = fleet
        self.rides = rides
        self.bonus = bonus
        self.steps = steps

    @staticmethod
    def distance(a: int, b: int, x: int, y: int) -> int:
        return abs(x - a) + abs(y - b)
    
    @classmethod
    def ride_length(cls, ride: List[int]) -> int:
        return cls.distance(*ride[0:4])
    
    @classmethod
    def can_reach(cls, x: int, y: int, step: int, ride: List[int]) -> bool:
        return cls.distance(x, y, ride[0], ride[1]) + step <= ride[5] - cls.ride_length(ride)
    
    @property
    def max_bound(self):
        if not hasattr(self, '_max_bound'):
            self._max_bound = len(self.rides) * self.bonus \
                    + np.sum(np.apply_along_axis(lambda x: self.ride_length(x), 1, self.rides[:, 0:4]))
        
        return self._max_bound
    
    @property
    def rides_by_starting(self) -> List[List[int]]:
        return [k for k, ride in sorted(enumerate(self.rides), key=lambda x: x[1][4])]
    
    @classmethod
    def from_file(cls, file_path, *args, **kwargs):
        with open(file_path) as f:
            rows, cols, fleet, num_rides, bonus, steps = [int(i) for i in f.readline().split()]
            rides = np.array([[int(x) for x in line.strip().split()] for i, line in zip(range(num_rides), f.readlines())])

        return cls(rows, cols, fleet, rides, bonus, steps)

    def steps_for_rides(self, rides) -> int:
        x, y = 0, 0
        steps = 0
        for ride in [self.rides[r] for r in rides]:
            steps += max(ride[4], steps + problem.distance(x, y, ride[0], ride[1]))  # Move to ride start point
            steps += problem.ride_length(ride)  # Move to ride end point
            x, y = ride[2:4]
            
        return steps
    
    def is_car_valid(self, rides) -> bool:
        x, y = 0, 0
        step = 0
        for ride_index in rides:
            # Remove ride index
            ride = self.rides[ride_index]

            # Check if ride is valid
            if not self.can_reach(x, y, step, ride):
                return False

            # Update iterative variables
            step += max(ride[4], step + self.distance(x, y, ride[0], ride[1]))  # Move to ride start point
            step += self.ride_length(ride)  # Move to ride end point
            x, y = ride[2:4]

        return True

    def is_individual_valid(self, individual) -> bool:
        rides = [r for i in individual.values() for r in i]
        
        if len(rides) > len(set(rides)):
            return False
        
        for car, rides in individual.items():
            if not self.is_car_valid(rides):
                return False

        return True
    
    def __str__(self):
        return '\n'.join([f'Ride(x={i[0], i[1]}, y={i[2], i[3]}, starting={i[4]}, finish={i[5]})' for i in self.rides])

    def __repr__(self):
        s = f'Problem{{rows={self.rows}, cols={self.cols}, fleet={self.fleet}, bonus={self.bonus}, steps={self.steps}}}'
        if len(self.rides) < 20:
            s += f'\n{str(self)}'
        return s

### Problem instance

In [3]:
#problem = Problem.from_file('input/a_example.in')
problem = Problem.from_file('input/b_should_be_easy.in')
#problem = Problem.from_file('input/c_no_hurry.in')
#problem = Problem.from_file('input/d_metropolis.in')
#problem = Problem.from_file('input/e_high_bonus.in')
problem

Problem{rows=10000, cols=10000, fleet=400, bonus=2, steps=50000}

### Class that contains a Solution

The solution to this problem will be represented by a class that can be written and represented, so that following methods must be implemented:
* Serialize the solution into a string (`__str__`).
* Create a string that represents the solution (`__repr__`).
* Write to file the solution (`to_file`).

In [4]:
class Solution:
    """
    A solution for this problem that consists of a structure [[r1, r2, ..., rn]...]
    """
    def __init__(self, data):
        self.data = data

    def to_file(self, file_path):
        with open(file_path, 'w') as f:
            f.write(self.__str__())

    def __str__(self):
        return '\n'.join([' '.join([str(len(rides))] + [str(r) for r in rides]) for rides in self.data.values()])

    def __repr__(self):
        return self.__str__()

## Genetic Algorithm's Operators

In order to generate a population for the Genetic Algorithm it's necessary to define following constants:
* `INDIVIDUAL_TYPE`: The base type for an individual.
* `INDIVIDUAL_SIZE`: Initial size of the individual.

Running the algorithm consists of generating and evaluating individuals, so it's required to define functions for:
* Generating a new individual based on problem instance (`generate`).
* Evaluating the current fitness value of the individual (`evaluate`).

If it's necessary, following functions can be defined:
* Mutate individuals (`mutate`).
* Crossover (`crossover`).

### Helpers

In [5]:
def find_itinerary(car: int, problem: Problem, rides: List[int]) -> List[int]:
    itinerary = []
    step = 0
    x, y = (0, 0)
    try:
        candidates = [i for i in rides if problem.can_reach(x, y, step, problem.rides[i])]
        while len(candidates) and step <= problem.steps:
            # Get a valid ride and remove it from rides pool
            ride_index = random.choice(candidates)
            candidates.remove(ride_index)
            
            # Add ride to itinerary
            itinerary.append(ride_index)
            ride = problem.rides[ride_index]
            
            # Update iterative variables
            x, y = ride[2:4]
            step += max(ride[4], step + problem.distance(x, y, ride[0], ride[1]))  # Move to ride start point
            step += problem.ride_length(ride)  # Move to ride end point
            candidates = [i for i in candidates if problem.can_reach(x, y, step, problem.rides[i])]
    except StopIteration:
        pass
        
    return itinerary

### Constants

In [6]:
INDIVIDUAL_TYPE = dict

### Generation

In [7]:
def generate(problem: Problem) -> Dict[int, List[int]]:
    individual = OrderedDict({})
    rides_completed = []
    rides_available = problem.rides_by_starting
    for c in range(problem.fleet):
        rides_available = [i for i in rides_available if i not in rides_completed]
        rides = find_itinerary(c, problem, rides_available)
        individual[c] = rides
        rides_completed += rides
        
    return creator.Individual(individual)


def generate_population(n: int) -> List:
    pool = multiprocessing.Pool()
    tasks = [pool.apply_async(toolbox.individual) for _ in range(n)]
    return [t.get() for t in tasks]
    

### Evaluation

In [8]:
def evaluate(individual, problem: Problem) -> Tuple[float]:
    if problem.is_individual_valid(individual):
        rides = np.array([problem.rides[r] for car, rides in individual.items() for r in rides])
        distances = np.sum(np.apply_along_axis(problem.ride_length, 1, rides))
        bonus = 0
        return float(distances + bonus),
    else:
        return 0.0,

### Mutation

In [9]:
def mutate(individual, problem) -> Tuple:
    car = random.choice(list(individual.keys()))
    ride_index = random.randint(0, len(individual[car])-1)
    ride = individual[car][ride_index]
    x, y = (0, 0) if ride_index == 0 else problem.rides[individual[car][ride_index-1]][2:4]
    rides_taken = {i for ride in individual.values() for i in ride}
    rides_available = [r for r in problem.rides_by_starting if r not in rides_taken and problem.can_reach(x, y, 0, problem.rides[r])]
    
    if rides_available:
        individual[car][ride_index] = random.choice(rides_available)
    else:
        del individual[car][ride_index]
        
    return individual,

### Crossover

In [10]:
def crossover_by_starting(ind1, ind2, problem: Problem) -> Tuple:
    for car in ind1:
        size = min(len(ind1[car]), len(ind2[car]))
        if size > 1:
            cxpoint1 = random.randint(1, size-1)
            ride = problem.rides[ind1[car][cxpoint1]]
            prev_ride = problem.rides[ind1[car][cxpoint1-1]]
            steps = problem.steps_for_rides(ind1[car][:cxpoint1])

            cxpoint2 = None
            for i, r in ((i, r) for i, r in enumerate(ind2[car]) if cxpoint2 is None):
                if problem.can_reach(prev_ride[2], prev_ride[3], steps, problem.rides[r]):
                    cxpoint2 = i

            if cxpoint2:
                ind1[car][cxpoint1:], ind2[car][cxpoint2:] = ind2[car][cxpoint2:], ind1[car][cxpoint1:]
        
    return ind1, ind2

## DEAP toolbox

### Types

In [11]:
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", INDIVIDUAL_TYPE, fitness=creator.Fitness)

### Population

In [12]:
toolbox = base.Toolbox()
toolbox.register("individual", generate, problem=problem)
toolbox.register("population", generate_population)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)

### Genetic functions

In [13]:
toolbox.register("evaluate", evaluate, problem=problem)
toolbox.register("mate", crossover_by_starting, problem=problem)
toolbox.register("mutate", mutate, problem=problem)
toolbox.register("select", tools.selNSGA2)

### Multiprocessing

In [14]:
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

## Solution

### Hyperparameters

In [15]:
GENERATIONS = 500
MU = 10
LAMBDA = 20
CROSSOVER_PROBABILITY = 0.5
MUTATION_PROBABILITY = 0.4

### Run the algorithm

In [ ]:
population = toolbox.population(n=MU)

hof = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

# population, log = algorithms.eaSimple(population, toolbox, CROSSOVER_PROBABILITY, MUTATION_PROBABILITY, GENERATIONS, stats, halloffame=hof)
population, log = algorithms.eaMuPlusLambda(population, toolbox, MU, LAMBDA, CROSSOVER_PROBABILITY, MUTATION_PROBABILITY, GENERATIONS, stats, halloffame=hof)
# population, log = algorithms.eaMuCommaLambda(population, toolbox, MU, LAMBDA, CROSSOVER_PROBABILITY, MUTATION_PROBABILITY, GENERATIONS, stats, halloffame=hof)

### Best individual

In [ ]:
solution = Solution(data=INDIVIDUAL_TYPE(hof[0]))
print(f'Best individual fitness: {int(hof[0].fitness.wvalues[0])}/{problem.max_bound}')
# print(str(solution))

### Plot evolution

In [ ]:
gen, avg, min_, max_ = log.select("gen", "avg", "min", "max")
plt.plot(gen, avg, label="avg")
plt.plot(gen, min_, label="min")
plt.plot(gen, max_, label="max")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="lower right")
plt.show()

In [ ]:
#solution.to_file('output/a.txt')
solution.to_file('output/b.txt')
#solution.to_file('output/c.txt')
#solution.to_file('output/d.txt')
#solution.to_file('output/e.txt')